In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio import Affine
from shapely.geometry import Point
from rasterstats import point_query
#import xgboost as xgb
from sklearn import ensemble
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
import math
#from boruta import BorutaPy
from scipy.stats import norm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Predictor selection
predictors_options = ['LCZ', 'SM', 'NDVI', 'NDLI', 'IMD', 'SWIR1', 'SWIR2', 'SVF', 'DTM', 'CH']
checkboxes = [widgets.Checkbox(value=True, description=option) for option in predictors_options]
checkbox_widget = widgets.VBox(checkboxes)
display(checkbox_widget)

# Normalization selection
normalization_w = widgets.RadioButtons(
    options=['YES', 'NO'],
    description='Use normalized predictors?',
    style={'description_width': 'initial'}
)
display(normalization_w)

RadioButtons(description='Use normalized predictors?', options=('YES', 'NO'), style=DescriptionStyle(descripti…

In [6]:
# Extract once
selected_predictors = [cb.description for cb in checkboxes if cb.value]
normalization = normalization_w.value

# Without Feature Selection

In [ ]:
def run_rf_pipeline(month, hour, predictors, normalization):

    print(f"Running pipeline for {month} - {hour}...")

    # Load temperature data
    #data = f'../Data/CML/HW_extracted/HW_{month}_2022/CML-ARPA_HW_{month}_2022_{hour}.csv'
    data = f'../Data/CML/HW_extracted/alt_corr/CML-ARPA_HW_{month}_2022_{hour}.csv'
    data_df = pd.read_csv(data)

    geometry = [Point(xy) for xy in zip(data_df['long'], data_df['lat'])]
    data_gdf = gpd.GeoDataFrame(data_df, geometry=geometry, crs='EPSG:4326')
    data_gdf = data_gdf.to_crs(epsg=32632)

    # Merge LCZ and train/test labels
    data_class_df = pd.read_csv('../Data/LCZ/ARPA_CML_stations_LCZ.csv', delimiter=',')
    data_class_df = data_class_df[['station', 'dominant_class', 'train/test']]
    data_class_df.rename(columns={'dominant_class': 'LCZ'}, inplace=True)

    merged_df = pd.merge(data_gdf, data_class_df, on='station', how='inner')
    data_train = merged_df.loc[merged_df['train/test'] == 'train'].copy()
    data_test = merged_df.loc[merged_df['train/test'] == 'test'].copy()

    # Select predictor paths
    predictors_dict = {
        'NDVI': f'../Data/NDVI/NDVI_2022_{month}_mean.tif',
        'LCZ': f'../Data/LCZ/LCZ_{month}_20m_majority.tif',
        'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_mean.tif',
        'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_mean.tif',
        'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_mean.tif',
        'SVF': '../Data/SVF/SVF_Final_20m_mean.tif',
        'DTM': '../Data/Elevation/Elevation_20m_mean.tif',
        'SM': f'../Data/Soil_Moisture/Soil_Moisture_2022_{month}_mean.tif',
        'CH': '../Data/Canopy_Heights_ETH/Canopy_Heights_ETH_20m_norm_mean.tif'
    }

    if normalization == 'YES':
        predictors_dict.update({
            'LCZ': f'../Data/LCZ/LCZ_{month}_20m_norm_majority.tif',
            'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_norm_mean.tif',
            'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_norm_mean.tif',
            'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_norm_mean.tif',
            'DTM': '../Data/Elevation/Elevation_20m_norm_mean.tif',
        })

    predictors_dict = {k: v for k, v in predictors_dict.items() if k in predictors}
    
    # Sample predictors
    for pred in predictors_dict:
        raster = predictors_dict[pred]
        data_train[pred] = point_query(data_train.geometry, raster, nodata=-999)
        data_test[pred] = point_query(data_test.geometry, raster, nodata=-999)

    # Train/Test split
    X_train = data_train[predictors]
    y_train = data_train['avg_temp']
    X_test = data_test[predictors]
    y_test = data_test['avg_temp']

    # Train model (Here uncomment in the case of defined HP or default HP)
    # regr = ensemble.RandomForestRegressor(
    # n_estimators=200,
    # max_depth=10,         # limit tree depth
    # min_samples_leaf=5,   # reduce variance
    # max_features='sqrt',  # prevent over-reliance on a few features
    # random_state=42
    # )
    # For default HP
    regr = ensemble.RandomForestRegressor(random_state=42)
    regr.fit(X_train, y_train)

    # Predict on test set
    y_pred = regr.predict(X_test)
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    rmse = round(math.sqrt(mean_squared_error(y_test, y_pred)), 3)
    r2_test = round(r2_score(y_test, y_pred), 3)

    # NEW: Predict on train set
    y_train_pred = regr.predict(X_train)
    mae_train = round(mean_absolute_error(y_train, y_train_pred), 3)
    rmse_train = round(math.sqrt(mean_squared_error(y_train, y_train_pred)), 3)
    r2_train = round(r2_score(y_train, y_train_pred), 3)

    # Feature importance
    feat_imp = dict(zip(predictors, regr.feature_importances_))

    # Print summary
    print(f"Done → TEST → MAE: {mae}, RMSE: {rmse}, R2: {r2_test}")
    print(f"         TRAIN → MAE: {mae_train}, RMSE: {rmse_train}, R2: {r2_train}")

    raster_arrays = []
    for pred in predictors:
        with rasterio.open(predictors_dict[pred]) as src:
            array = src.read(1).astype(np.float32)
            raster_arrays.append(array)
            if pred == predictors[0]:  # Use first raster for metadata
                meta = src.meta.copy()

    stack = np.stack(raster_arrays, axis=0)  # shape: (num_bands, height, width)
    n_bands, height, width = stack.shape
    flat_stack = stack.reshape(n_bands, -1).T  # shape: (pixels, bands)

    # Predict on full raster
    pred_full = regr.predict(flat_stack)
    pred_raster = pred_full.reshape(height, width)

    # Save prediction to GeoTIFF
    meta.update({
        'count': 1,
        'dtype': 'float32'
    })
    #out_path = f'../Data/Predicted_maps/Automated/no_SM/no_FS_defined_HP/red_rf_{month}_{hour}.tif'
    out_path = f'../Data/Predicted_maps/Automated/alt_corr/no_SM/no_FS_default_HP/pred_rf_{month}_{hour}.tif'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(pred_raster.astype(np.float32), 1)

    print(f"Prediction raster saved to {out_path}")
    
    # Save results
    result = {
        'month': month,
        'hour': hour,
        'MAE_test': mae,
        'RMSE_test': rmse,
        'R2_test': r2_test,
        'MAE_train': mae_train,
        'RMSE_train': rmse_train,
        'R2_train': r2_train,
        **feat_imp  # this unpacks the feature importances
    }

    return result

In [ ]:
results_all = []
months = ['Feb', 'May', 'June', 'July', 'Oct']
hours = ['HP', 'WP', 'CP1', 'CP2', 'MUHI']

for m in months:
    for h in hours:
        result = run_rf_pipeline(m, h, selected_predictors, normalization)
        results_all.append(result)

In [ ]:
# Save to DataFrame
results_df = pd.DataFrame(results_all)
results_df.to_csv('../Data/Predicted_maps/Automated/alt_corr/no_DTM/no_FS_default_HP/RF_results.csv', index=False)

results_df.head()

In [ ]:
pwd

# Test with feature selection

**Here we also try split of 50/50**

In [ ]:
def run_rf_pipeline(month, hour, predictors, normalization):

    print(f"Running pipeline for {month} - {hour}...")

    # Load temperature data. Here uncomment when needed
    #data = f'../Data/CML/HW_extracted/HW_{month}_2022/CML-ARPA_HW_{month}_2022_{hour}.csv'
    data = f'../Data/CML/HW_extracted/alt_corr/CML-ARPA_HW_{month}_2022_{hour}.csv'
    #data = f'../Data/CML/HW_extracted/meteotracker/22052022.csv'
    data_df = pd.read_csv(data)

    geometry = [Point(xy) for xy in zip(data_df['long'], data_df['lat'])]
    data_gdf = gpd.GeoDataFrame(data_df, geometry=geometry, crs='EPSG:4326')
    data_gdf = data_gdf.to_crs(epsg=32632)

    # Merge LCZ and train/test labels
    data_class_df = pd.read_csv('../Data/LCZ/ARPA_CML_stations_LCZ.csv', delimiter=',')
    data_class_df = data_class_df[['station', 'dominant_class', 'train/test']]
    data_class_df.rename(columns={'dominant_class': 'LCZ'}, inplace=True)

    merged_df = pd.merge(data_gdf, data_class_df, on='station', how='inner')
    data_train = merged_df.loc[merged_df['train/test'] == 'train'].copy()
    data_test = merged_df.loc[merged_df['train/test'] == 'test'].copy()
    # HERE WE ADD PART WITH 50/50 SPLIT
    # data_train, data_test = train_test_split(
    #     merged_df,
    #     test_size=0.5,
    #     random_state=42
    # )

    # Select predictor paths
    predictors_dict = {
        'NDVI': f'../Data/NDVI/NDVI_2022_{month}_mean.tif',
        'LCZ': f'../Data/LCZ/LCZ_{month}_20m_majority.tif',
        'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_mean.tif',
        'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_mean.tif',
        'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_mean.tif',
        'SVF': '../Data/SVF/SVF_Final_20m_mean.tif',
        'DTM': '../Data/Elevation/Elevation_20m_mean.tif',
        'SM': f'../Data/Soil_Moisture/Soil_Moisture_2022_{month}_mean.tif',
        'CH': '../Data/Canopy_Heights_ETH/Canopy_Heights_ETH_20m_norm_mean.tif'
    }

    if normalization == 'YES':
        predictors_dict.update({
            'LCZ': f'../Data/LCZ/LCZ_{month}_20m_norm_majority.tif',
            'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_norm_mean.tif',
            'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_norm_mean.tif',
            'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_norm_mean.tif',
            'DTM': '../Data/Elevation/Elevation_20m_norm_mean.tif',
        })

    predictors_dict = {k: v for k, v in predictors_dict.items() if k in predictors}

    # Sample predictors
    for pred in predictors_dict:
        raster = predictors_dict[pred]
        data_train[pred] = point_query(data_train.geometry, raster, nodata=-999)
        data_test[pred] = point_query(data_test.geometry, raster, nodata=-999)

    # Drop rows with any NaNs in predictors
    data_train = data_train.dropna(subset=predictors)
    data_test = data_test.dropna(subset=predictors)

    # Use all predictors initially
    X_train_full = data_train[predictors]
    y_train = data_train['avg_temp']
    X_test_full = data_test[predictors]
    y_test = data_test['avg_temp']

    # First train to get feature importances (Here uncomment in the case of defined predictors or default predictors)
    # regr_full = ensemble.RandomForestRegressor(
    #     n_estimators=200,
    #     max_depth=10,
    #     min_samples_leaf=5,
    #     max_features='sqrt',
    #     random_state=42
    # )
    regr_full = ensemble.RandomForestRegressor(random_state=42)
    regr_full.fit(X_train_full, y_train)

    # Select top N features
    N = 5
    importances = regr_full.feature_importances_
    feature_importance_dict = dict(zip(predictors, importances))
    selected_features = sorted(feature_importance_dict, key=feature_importance_dict.get, reverse=True)[:N]
    print(f"Selected features for {month}-{hour}: {selected_features}")

    # Re-train model on selected features (Here uncomment in the case of defined predictors or default predictors)
    X_train = data_train[selected_features]
    X_test = data_test[selected_features]
    # regr = ensemble.RandomForestRegressor(
    #     n_estimators=200,
    #     max_depth=10,
    #     min_samples_leaf=5,
    #     max_features='sqrt',
    #     random_state=42
    # )
    regr = ensemble.RandomForestRegressor(random_state=42)
    regr.fit(X_train, y_train)

    # Predict on test
    y_pred = regr.predict(X_test)
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    rmse = round(math.sqrt(mean_squared_error(y_test, y_pred)), 3)
    r2_test = round(r2_score(y_test, y_pred), 3)

    # Predict on train
    y_train_pred = regr.predict(X_train)
    mae_train = round(mean_absolute_error(y_train, y_train_pred), 3)
    rmse_train = round(math.sqrt(mean_squared_error(y_train, y_train_pred)), 3)
    r2_train = round(r2_score(y_train, y_train_pred), 3)

    # Feature importance (from final model)
    feat_imp = dict(zip(selected_features, regr.feature_importances_))

    print(f"Done → TEST → MAE: {mae}, RMSE: {rmse}, R2: {r2_test}")
    print(f"       TRAIN → MAE: {mae_train}, RMSE: {rmse_train}, R2: {r2_train}")

    # Predict full raster
    raster_arrays = []
    for pred in selected_features:
        with rasterio.open(predictors_dict[pred]) as src:
            array = src.read(1).astype(np.float32)
            raster_arrays.append(array)
            if pred == selected_features[0]:
                meta = src.meta.copy()

    stack = np.stack(raster_arrays, axis=0)
    n_bands, height, width = stack.shape
    flat_stack = stack.reshape(n_bands, -1).T

    pred_full = regr.predict(flat_stack)
    pred_raster = pred_full.reshape(height, width)

    #Save prediction raster
    meta.update({'count': 1, 'dtype': 'float32'})
    #out_path = f'../Data/Predicted_maps/Automated/no_SM_LCZ/with_FS_defined_HP/pred_rf_{month}_{hour}.tif'
    out_path = f'../Data/Predicted_maps/Automated/alt_corr/no_SM/with_FS_default_HP/pred_rf_{month}_{hour}.tif'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(pred_raster.astype(np.float32), 1)

    print(f"Prediction raster saved to {out_path}")

    result = {
        'month': month,
        'hour': hour,
        'MAE_test': mae,
        'RMSE_test': rmse,
        'R2_test': r2_test,
        'MAE_train': mae_train,
        'RMSE_train': rmse_train,
        'R2_train': r2_train,
        **feat_imp
    }

    return result, selected_features

In [ ]:
results_all = []
selected_feats_all = []
months = ['Feb', 'May', 'June', 'July', 'Oct']
#months = ['May']
hours = ['HP', 'WP', 'CP1', 'CP2', 'MUHI']
#hours = ['WP']

for m in months:
    for h in hours:
        result, selected_feats = run_rf_pipeline(m, h, selected_predictors, normalization)
        results_all.append(result)
        selected_feats_all.append({
            "month": m,
            "hour": h,
            "selected_features": selected_feats
        })

In [ ]:
# Save to DataFrame
results_df = pd.DataFrame(results_all)
results_df.to_csv('../Data/Predicted_maps/Automated/alt_corr/no_SM/with_FS_default_HP/RF_results_withFS_defaultHP.csv', index=False)

results_df.head()

# With CV

In [7]:
def run_rf_pipeline(month, hour, predictors, normalization):

    print(f"Running pipeline for {month} - {hour}...")

    # Load temperature data. Here uncomment when needed
    #data = f'../Data/CML/HW_extracted/HW_{month}_2022/CML-ARPA_HW_{month}_2022_{hour}.csv'
    data = f'../Data/CML/HW_extracted/alt_corr1/CML-ARPA_HW_{month}_2022_{hour}.csv'
    #data = f'../Data/CML/HW_extracted/meteotracker/22052022.csv'
    data_df = pd.read_csv(data)

    geometry = [Point(xy) for xy in zip(data_df['long'], data_df['lat'])]
    data_gdf = gpd.GeoDataFrame(data_df, geometry=geometry, crs='EPSG:4326')
    data_gdf = data_gdf.to_crs(epsg=32632)

    # Merge LCZ and train/test labels
    data_class_df = pd.read_csv('../Data/LCZ/ARPA_CML_stations_LCZ.csv', delimiter=',')
    #data_class_df = data_class_df[data_class_df['network']=='ARPA'] # Uncomment here to select only ARPA or CML stations
    data_class_df = data_class_df[['station', 'dominant_class', 'train/test', 'urban/natural']]
    data_class_df.rename(columns={'dominant_class': 'LCZ'}, inplace=True)

    merged_df = pd.merge(data_gdf, data_class_df, on='station', how='inner')
    data_train = merged_df.loc[merged_df['train/test'] == 'train'].copy()
    data_test = merged_df.loc[merged_df['train/test'] == 'test'].copy()

# ------- STRATIFIED SAMPLING -------
    # Uncomment when needed
    #STRATIFIED SPLIT by station and 'urban/natural'
    # station_meta = merged_df[['station', 'urban/natural']].drop_duplicates()

    # train_stations, test_stations = train_test_split(
    #     station_meta,
    #     test_size=0.3,  # adjust test size as needed
    #     stratify=station_meta['urban/natural'],
    #     random_state=42
    # )
    
    # data_train = merged_df[merged_df['station'].isin(train_stations['station'])].copy()
    # data_test = merged_df[merged_df['station'].isin(test_stations['station'])].copy()

# -------------------

    # Select predictor paths
    # predictors_dict = {
    #     'NDVI': f'../Data/NDVI/NDVI_2022_{month}_mean.tif',
    #     'NDLI': f'../Data/NDLI/NDLI_2022_{month}_mean.tif',
    #     'LCZ': f'../Data/LCZ/LCZ_{month}_20m_majority.tif',
    #     'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_mean.tif',
    #     'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_mean.tif',
    #     'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_mean.tif',
    #     'SVF': '../Data/SVF/SVF_Final_20m_mean.tif',
    #     'DTM': '../Data/Elevation/Elevation_20m_mean.tif',
    #     'SM': f'../Data/Soil_Moisture/Soil_Moisture_2022_{month}_mean.tif',
    #     'CH': '../Data/Canopy_Heights_ETH/Canopy_Heights_ETH_20m_norm_mean.tif'
    # }

    # Predictors with a buffer of 150m around
    predictors_dict = {
        'NDVI': f'../Data/NDVI/NDVI_2022_{month}_mean150.tif',
        'NDLI': f'../Data/NDLI/NDLI_2022_{month}_mean150.tif',
        'LCZ': f'../Data/LCZ/LCZ_{month}_20m_majority150.tif',
        'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_mean150.tif',
        'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_mean150.tif',
        'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_mean150.tif',
        'SVF': '../Data/SVF/SVF_Final_20m_mean150.tif',
        'DTM': '../Data/Elevation/Elevation_20m_mean150.tif',
        'SM': f'../Data/Soil_Moisture/Soil_Moisture_2022_{month}_mean150.tif',
        'CH': '../Data/Canopy_Heights_ETH/Canopy_Heights_ETH_20m_mean150.tif'
    }

    if normalization == 'YES':
        predictors_dict.update({
            'LCZ': f'../Data/LCZ/LCZ_{month}_20m_norm_majority.tif',
            'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_norm_mean.tif',
            'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_norm_mean.tif',
            'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_norm_mean.tif',
            'DTM': '../Data/Elevation/Elevation_20m_norm_mean.tif',
        })
    
    predictors_dict = {k: v for k, v in predictors_dict.items() if k in predictors}
    selected_features = list(predictors_dict.keys())

    # Sample predictors
    for pred in predictors_dict:
        raster = predictors_dict[pred]
        data_train[pred] = point_query(data_train.geometry, raster, nodata=-999)
        data_test[pred] = point_query(data_test.geometry, raster, nodata=-999)

    # Drop rows with any NaNs in predictors
    data_train = data_train.dropna(subset=predictors)
    data_test = data_test.dropna(subset=predictors)
    
    # Train/Test split
    X_train = data_train[predictors]
    y_train = data_train['avg_temp']
    X_test = data_test[predictors]
    y_test = data_test['avg_temp']

    # --- Hyperparameter Tuning ---
    param_dist = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 30, None],
        'max_features': ['sqrt', 0.8, 1.0],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }

    rf = RandomForestRegressor(random_state=42)

    search = RandomizedSearchCV(
        rf,
        param_distributions=param_dist,
        n_iter=20,
        scoring='neg_mean_squared_error',
        cv=5,
        n_jobs=-1,
        verbose=0,
        random_state=42
    )
    search.fit(X_train, y_train)
    best_params = search.best_params_

    print(f"Best hyperparameters for {month}-{hour}: {best_params}")

    # Train final model using best hyperparameters
    regr = RandomForestRegressor(**best_params, random_state=42)
    regr.fit(X_train, y_train)

    # --- Prediction and Evaluation ---
    y_pred = regr.predict(X_test)
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    rmse = round(math.sqrt(mean_squared_error(y_test, y_pred)), 3)
    r2_test = round(r2_score(y_test, y_pred), 3)

    y_train_pred = regr.predict(X_train)
    mae_train = round(mean_absolute_error(y_train, y_train_pred), 3)
    rmse_train = round(math.sqrt(mean_squared_error(y_train, y_train_pred)), 3)
    r2_train = round(r2_score(y_train, y_train_pred), 3)

    # Feature importance (from final model)
    feat_imp = dict(zip(selected_features, regr.feature_importances_))

    print(f"Done → TEST → MAE: {mae}, RMSE: {rmse}, R2: {r2_test}")
    print(f"       TRAIN → MAE: {mae_train}, RMSE: {rmse_train}, R2: {r2_train}")

    # Predict full raster
    raster_arrays = []
    for pred in selected_features:
        with rasterio.open(predictors_dict[pred]) as src:
            array = src.read(1).astype(np.float32)
            raster_arrays.append(array)
            if pred == selected_features[0]:
                meta = src.meta.copy()

    stack = np.stack(raster_arrays, axis=0)
    n_bands, height, width = stack.shape
    flat_stack = stack.reshape(n_bands, -1).T

    pred_full = regr.predict(flat_stack)
    pred_raster = pred_full.reshape(height, width)

    #Save prediction raster
    meta.update({'count': 1, 'dtype': 'float32'})
    #out_path = f'../Data/Predicted_maps/Automated/no_SM_LCZ/with_FS_defined_HP/pred_rf_{month}_{hour}.tif'
    out_path = f'../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150/pred_rf_{month}_{hour}.tif'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(pred_raster.astype(np.float32), 1)

    print(f"Prediction raster saved to {out_path}")

    result = {
        'month': month,
        'hour': hour,
        'MAE_test': mae,
        'RMSE_test': rmse,
        'R2_test': r2_test,
        'MAE_train': mae_train,
        'RMSE_train': rmse_train,
        'R2_train': r2_train,
        **dict(zip(predictors, regr.feature_importances_))
    }

    return result, selected_features, best_params

In [8]:
results_all = []
selected_feats_all = []
best_params_all = []

months = ['Feb', 'May', 'June', 'July', 'Oct']
hours = ['HP', 'WP', 'CP1', 'CP2', 'MUHI']

for m in months:
    for h in hours:
        print(f"\n>>> Running for {m} - {h}")
        # Assuming selected_predictors and normalization are defined outside
        result, selected_features, best_params = run_rf_pipeline(m, h, selected_predictors, normalization)
        
        results_all.append(result)
        selected_feats_all.append({
            "month": m,
            "hour": h,
            "selected_features": selected_features
        })
        best_params_all.append({
            "month": m,
            "hour": h,
            **best_params
        })


>>> Running for Feb - HP
Running pipeline for Feb - HP...
Best hyperparameters for Feb-HP: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}
Done → TEST → MAE: 0.55, RMSE: 0.643, R2: 0.137
       TRAIN → MAE: 0.239, RMSE: 0.306, R2: 0.886


C:\Users\Matej\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Prediction raster saved to ../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150_CV-100/pred_rf_Feb_HP.tif

>>> Running for Feb - WP
Running pipeline for Feb - WP...
Best hyperparameters for Feb-WP: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}
Done → TEST → MAE: 0.546, RMSE: 0.738, R2: -0.241
       TRAIN → MAE: 0.315, RMSE: 0.44, R2: 0.611


C:\Users\Matej\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Prediction raster saved to ../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150_CV-100/pred_rf_Feb_WP.tif

>>> Running for Feb - CP1
Running pipeline for Feb - CP1...
Best hyperparameters for Feb-CP1: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}
Done → TEST → MAE: 0.622, RMSE: 0.824, R2: -0.492
       TRAIN → MAE: 0.316, RMSE: 0.383, R2: 0.692


...

Prediction raster saved to ../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150_CV-100/pred_rf_Oct_CP1.tif

>>> Running for Oct - CP2
Running pipeline for Oct - CP2...
Best hyperparameters for Oct-CP2: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1.0, 'max_depth': 20}
Done → TEST → MAE: 0.972, RMSE: 1.212, R2: 0.179
       TRAIN → MAE: 0.415, RMSE: 0.53, R2: 0.848


C:\Users\Matej\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Prediction raster saved to ../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150_CV-100/pred_rf_Oct_CP2.tif

>>> Running for Oct - MUHI
Running pipeline for Oct - MUHI...
Best hyperparameters for Oct-MUHI: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 1.0, 'max_depth': 30}
Done → TEST → MAE: 0.8, RMSE: 0.936, R2: 0.196
       TRAIN → MAE: 0.276, RMSE: 0.338, R2: 0.906


C:\Users\Matej\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Prediction raster saved to ../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150_CV-100/pred_rf_Oct_MUHI.tif


In [10]:
results_df = pd.DataFrame(results_all)
results_df.to_csv('../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150/RF_results.csv', index=False)

selected_feats_df = pd.DataFrame(selected_feats_all)
selected_feats_df.to_csv('../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150/RF_selected_feats.csv', index=False)

best_params_df = pd.DataFrame(best_params_all)
best_params_df.to_csv('../Data/Predicted_maps/Automated/alt_corr1/with_CV/no_SM_no_DTM/no_FS_best_HP_buffer_150/RF_best_params.csv', index=False)

print("All results saved.")

All results saved.


# With Recursive Feature Elimination (RFE) Feature Selection

In [ ]:
def run_rf_pipeline(month, hour, predictors, normalization):

    print(f"Running pipeline for {month} - {hour}...")

    # Load temperature data. Here uncomment when needed
    #data = f'../Data/CML/HW_extracted/HW_{month}_2022/CML-ARPA_HW_{month}_2022_{hour}.csv'
    data = f'../Data/CML/HW_extracted/alt_corr/CML-ARPA_HW_{month}_2022_{hour}.csv'
    #data = f'../Data/CML/HW_extracted/meteotracker/22052022.csv'
    data_df = pd.read_csv(data)

    geometry = [Point(xy) for xy in zip(data_df['long'], data_df['lat'])]
    data_gdf = gpd.GeoDataFrame(data_df, geometry=geometry, crs='EPSG:4326')
    data_gdf = data_gdf.to_crs(epsg=32632)

    # Merge LCZ and train/test labels
    data_class_df = pd.read_csv('../Data/LCZ/ARPA_CML_stations_LCZ.csv', delimiter=',')
    #data_class_df = data_class_df[data_class_df['network']=='ARPA'] # Uncomment here to select only ARPA or CML stations
    data_class_df = data_class_df[['station', 'dominant_class', 'train/test', 'urban/natural']]
    data_class_df.rename(columns={'dominant_class': 'LCZ'}, inplace=True)

    merged_df = pd.merge(data_gdf, data_class_df, on='station', how='inner')
    data_train = merged_df.loc[merged_df['train/test'] == 'train'].copy()
    data_test = merged_df.loc[merged_df['train/test'] == 'test'].copy()

    # Select predictor paths
    predictors_dict = {
        'NDVI': f'../Data/NDVI/NDVI_2022_{month}_mean.tif',
        'NDLI': f'../Data/NDLI/NDLI_2022_{month}_mean.tif',
        'LCZ': f'../Data/LCZ/LCZ_{month}_20m_majority.tif',
        'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_mean.tif',
        'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_mean.tif',
        'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_mean.tif',
        'SVF': '../Data/SVF/SVF_Final_20m_mean.tif',
        'DTM': '../Data/Elevation/Elevation_20m_mean.tif',
        'SM': f'../Data/Soil_Moisture/Soil_Moisture_2022_{month}_mean.tif',
        'CH': '../Data/Canopy_Heights_ETH/Canopy_Heights_ETH_20m_norm_mean.tif'
    }

    if normalization == 'YES':
        predictors_dict.update({
            'LCZ': f'../Data/LCZ/LCZ_{month}_20m_norm_majority.tif',
            'IMD': '../Data/IMD_Copernicus_2018/IMD_Copernicus_2018_20m_norm_mean.tif',
            'SWIR1': f'../Data/SWIR/SWIR1_2022_{month}_norm_mean.tif',
            'SWIR2': f'../Data/SWIR/SWIR2_2022_{month}_norm_mean.tif',
            'DTM': '../Data/Elevation/Elevation_20m_norm_mean.tif',
        })
    
    predictors_dict = {k: v for k, v in predictors_dict.items() if k in predictors}
    selected_features = list(predictors_dict.keys())

    # Sample predictors
    for pred in predictors_dict:
        raster = predictors_dict[pred]
        data_train[pred] = point_query(data_train.geometry, raster, nodata=-999)
        data_test[pred] = point_query(data_test.geometry, raster, nodata=-999)

    # Drop rows with any NaNs in predictors
    data_train = data_train.dropna(subset=predictors)
    data_test = data_test.dropna(subset=predictors)

    initial_features = selected_features
    X_train_raw = data_train[initial_features]
    y_train = data_train['avg_temp']

# ------- RFE feature selection -------

    base_rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    rfe = RFE(estimator=base_rf, n_features_to_select=5)
    rfe.fit(X_train_raw, y_train)
    
    # Selected features
    selected_features = list(np.array(initial_features)[rfe.support_])
    print(f"Selected features by RFE for {month}-{hour}: {selected_features}")

    # Feature ranking
    feature_ranking = pd.DataFrame({
        'feature': initial_features,
        'rank': rfe.ranking_,
        'selected': rfe.support_
    })
        
    # Train/Test split
    X_train = data_train[selected_features]
    y_train = data_train['avg_temp']
    X_test = data_test[selected_features]
    y_test = data_test['avg_temp']

# -------

    # --- Hyperparameter Tuning ---
    param_dist = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 30, None],
        'max_features': ['sqrt', 0.8, 1.0],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }

    rf = RandomForestRegressor(random_state=42)

    search = RandomizedSearchCV(
        rf,
        param_distributions=param_dist,
        n_iter=20,
        scoring='neg_mean_squared_error',
        cv=5,
        n_jobs=-1,
        verbose=0,
        random_state=42
    )
    search.fit(X_train, y_train)
    best_params = search.best_params_

    print(f"Best hyperparameters for {month}-{hour}: {best_params}")

    # Train final model using best hyperparameters
    regr = RandomForestRegressor(**best_params, random_state=42)
    regr.fit(X_train, y_train)

    # --- Prediction and Evaluation ---
    y_pred = regr.predict(X_test)
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    rmse = round(math.sqrt(mean_squared_error(y_test, y_pred)), 3)
    r2_test = round(r2_score(y_test, y_pred), 3)

    y_train_pred = regr.predict(X_train)
    mae_train = round(mean_absolute_error(y_train, y_train_pred), 3)
    rmse_train = round(math.sqrt(mean_squared_error(y_train, y_train_pred)), 3)
    r2_train = round(r2_score(y_train, y_train_pred), 3)

    # Feature importance (from final model)
    feat_imp = dict(zip(selected_features, regr.feature_importances_))

    print(f"Done → TEST → MAE: {mae}, RMSE: {rmse}, R2: {r2_test}")
    print(f"       TRAIN → MAE: {mae_train}, RMSE: {rmse_train}, R2: {r2_train}")

    # Predict full raster
    raster_arrays = []
    for pred in selected_features:
        with rasterio.open(predictors_dict[pred]) as src:
            array = src.read(1).astype(np.float32)
            raster_arrays.append(array)
            if pred == selected_features[0]:
                meta = src.meta.copy()

    stack = np.stack(raster_arrays, axis=0)
    n_bands, height, width = stack.shape
    flat_stack = stack.reshape(n_bands, -1).T

    pred_full = regr.predict(flat_stack)
    pred_raster = pred_full.reshape(height, width)

    #Save prediction raster
    meta.update({'count': 1, 'dtype': 'float32'})
    #out_path = f'../Data/Predicted_maps/Automated/no_SM_LCZ/with_FS_defined_HP/pred_rf_{month}_{hour}.tif'
    out_path = f'../Data/Predicted_maps/Automated/alt_corr/no_SM/RFE_FS_best_HP/pred_rf_{month}_{hour}.tif'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(pred_raster.astype(np.float32), 1)

    print(f"Prediction raster saved to {out_path}")

    result = {
        'month': month,
        'hour': hour,
        'MAE_test': mae,
        'RMSE_test': rmse,
        'R2_test': r2_test,
        'MAE_train': mae_train,
        'RMSE_train': rmse_train,
        'R2_train': r2_train,
        **dict(zip(selected_features, regr.feature_importances_))
    }

    return result, selected_features, best_params, feature_ranking

In [ ]:
results_all = []
selected_feats_all = []
best_params_all = []

months = ['Feb', 'May', 'June', 'July', 'Oct']
hours = ['HP', 'WP', 'CP1', 'CP2', 'MUHI']

for m in months:
    for h in hours:
        print(f"\n>>> Running for {m} - {h}")
        # Assuming selected_predictors and normalization are defined outside
        result, selected_features, best_params, feature_ranking = run_rf_pipeline(m, h, selected_predictors, normalization)
        
        results_all.append(result)
        selected_feats_all.append({
            "month": m,
            "hour": h,
            "selected_features": selected_features
        })
        best_params_all.append({
            "month": m,
            "hour": h,
            **best_params
        })

In [ ]:
results_df = pd.DataFrame(results_all)
results_df.to_csv('../Data/Predicted_maps/Automated/alt_corr/no_SM/RFE_FS_best_HP/RF_results.csv', index=False)

selected_feats_df = pd.DataFrame(selected_feats_all)
selected_feats_df.to_csv('../Data/Predicted_maps/Automated/alt_corr/no_SM/RFE_FS_best_HP/RF_selected_feats.csv', index=False)

best_params_df = pd.DataFrame(best_params_all)
best_params_df.to_csv('../Data/Predicted_maps/Automated/alt_corr/no_SM/RFE_FS_best_HP/RF_best_params.csv', index=False)

print("All results saved.")